In [63]:
import numpy as np 
import pandas as pd 
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/marlonfacey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
reviews=pd.read_csv("austin_reviews.csv")

In [17]:
reviews
reviews = reviews.dropna(axis=0, subset=['comments'])

In [38]:
len(reviews)

324348

In [18]:
# Read more data in
# import glob

# all_files = glob.glob("austin_reviews_2021_2019"+"/*.csv")
# li = []
# all_files[0]
# for filename in all_files:
#     df = pd.read_csv(filename)
#     li.append(df)
# listings_all = pd.concat(li, axis=0, ignore_index=True)

In [98]:
tokenized_reviews = []
for review in reviews['comments'][:50000]:
    tokenized_reviews.append(word_tokenize(review.lower()))

tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_reviews)]

In [124]:
d2v_model = Doc2Vec(tagged_data, vector_size = 256, window = 10, min_count = 1, epochs = 100)

In [125]:
# See similar reviews to the first review
test_doc = word_tokenize(reviews['comments'].iloc[0].lower())
test_doc_vector = d2v_model.infer_vector(test_doc)
closest_rev_idx = [i[0] for i in d2v_model.dv.most_similar(positive = [test_doc_vector])]
d2v_model.dv.most_similar(positive = [test_doc_vector])

[(0, 0.8040343523025513),
 (6029, 0.5176122784614563),
 (27972, 0.49575546383857727),
 (11510, 0.4631976783275604),
 (32741, 0.4590195119380951),
 (643, 0.45842665433883667),
 (36323, 0.4422696828842163),
 (9766, 0.44159334897994995),
 (36068, 0.432508260011673),
 (8167, 0.4319043755531311)]

In [126]:
print(reviews['comments'][0] + "\n")
for i in range(len(closest_rev_idx)):
    print(reviews['comments'][closest_rev_idx[i]] + "\n")


I stayed here during SXSW and had a really pleasant stay.  The house is a very relaxing environment and Paddy is both a friendly and professional host. 

I stayed here during SXSW and had a really pleasant stay.  The house is a very relaxing environment and Paddy is both a friendly and professional host. 

Alyson was very cordial and went above and beyond my expectations.  It's a must stay if you are in the Austin area.  Thanks Alyson!


The space is great for 1-2 travelers. I really liked the kitchen amenities and cutlery. You have all the utensils needed to prepare a meal. There's a stove-top, oven and toaster. The space is clean and efficient. One thing to keep in mind is the toilet seat is on a 1 foot high pedestal. The shower is also on an 8-inch pedestal and it's super tiny. Not a deal breaker but just something to keep in mind.

Transportation is not a problem because the bus stop is literally right outside the door. There are several restaurants about a 5 minute walk away and t

In [127]:
# Use K means cluster to get 2 clusters
k_means_model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=d2v_model.dv.vectors)
positive_cluster_center = k_means_model.cluster_centers_[0]
negative_cluster_center = k_means_model.cluster_centers_[1]

In [128]:
top_clust_0_rev_idx = [i[0] for i in d2v_model.dv.similar_by_vector(k_means_model.cluster_centers_[0], topn=10, restrict_vocab=None)]
d2v_model.dv.similar_by_vector(k_means_model.cluster_centers_[0], topn=10, restrict_vocab=None)

[(36282, 0.9728507399559021),
 (38204, 0.9706477522850037),
 (25598, 0.967058539390564),
 (47438, 0.9666666984558105),
 (37105, 0.9663744568824768),
 (2619, 0.9656502604484558),
 (28488, 0.9653806686401367),
 (42364, 0.9650701880455017),
 (36947, 0.9648461937904358),
 (15976, 0.9636886715888977)]

In [129]:
for i in range(len(top_clust_0_rev_idx)):
    print(reviews['comments'][top_clust_0_rev_idx[i]] + "\n")

Yvonne & Cameron's Airbnb backyard studio is well-appointed and charming.  It's within walking distance to downtown, and comes with a binder full of recommendations for places to eat and things to do.

Unfortunately, I didn't get to meet the hosts in person but assume had there been any problems, they would have been attentive.

The neighborhood was a bit more run-down than I'd expected, but I didn't feel unsafe and walked around alone at night without issue.

Great place within walking distance from downtown Austin. Clean place, helpful tips about the local area, very accommodating host

This studio apartment was very clean and inviting. Perfect for a couples getaway. Cute and artsy. Everything you need for a weekend getaway! Walking distance to cafes and shopping.

Gorgeous house that style wise looks very Austin-y with nice decor. There's a separate guest house in the back is very nice with it's own bathroom, two additional bedrooms in the main house share a bathroom. House had ever

In [130]:
top_clust_1_rev_idx = [i[0] for i in d2v_model.dv.similar_by_vector(k_means_model.cluster_centers_[1], topn=10, restrict_vocab=None)]
d2v_model.dv.similar_by_vector(k_means_model.cluster_centers_[1], topn=10, restrict_vocab=None)

[(36947, 0.974404513835907),
 (41452, 0.973639726638794),
 (33482, 0.9728446006774902),
 (16578, 0.9718385934829712),
 (30519, 0.9703684449195862),
 (11097, 0.9695250988006592),
 (32885, 0.9691112041473389),
 (46738, 0.9688645601272583),
 (47438, 0.9688200950622559),
 (47602, 0.967427134513855)]

In [131]:
for i in range(len(top_clust_1_rev_idx)):
    print(reviews['comments'][top_clust_1_rev_idx[i]] + "\n")

Steve hosted my boyfriend and I in his cottage for free during Hurricane Harvey. He was friendly and came to chat with Erroll and I as we settled in. I was a bit late on checking in and he was very responsive even waited outside for us when we ended up on the wrong street. I am very thankful for Steve and his family having us.

Had a good  stay here, lovely apartment and great location with plenty of bars and restaurants close by.

 Very large comfortable bed and good kitchen if you fancy cooking your own meals. Private off road parking too!  Bathroom has hairdryer and plenty of products and even a bathrobe each,   Very nice touches.

Donna's place is incredible! A tranquil and airy refuge that's walkable to downtown. Also close to great local bars, restaurants and shops. Totally safe and secure also. Will stay again!

Gigi’s space was wonderful! Comfortable, well appointed, clean, in a great neighborhood and so fun to stay in. The space was larger than I expected and she has such an e

In [135]:
cosine_similarity(k_means_model.cluster_centers_[0].reshape(1,-1), k_means_model.cluster_centers_[1].reshape(1,-1))

array([[0.93016183]], dtype=float32)

In [134]:
k_means_model.cluster_centers_

array([[-0.5986099 , -0.31769603,  0.4995188 ,  0.28278786,  0.17415886,
        -0.11144868, -0.7258697 , -0.17304061, -0.20619828,  0.5799234 ,
        -0.04509399, -0.3401889 , -0.14011809, -0.11513349,  0.41886052,
        -0.12381297, -0.11001544,  0.22070105, -0.15236868,  0.12374677,
         0.16093332, -0.17015655, -0.05132321, -0.12652276,  0.05472894,
         0.39514834,  0.77896005, -0.32559344,  0.40980738, -0.44987273,
        -0.5826855 , -0.2548483 ,  0.1562427 ,  0.06894238,  0.92911476,
        -0.12947446, -0.04243828,  0.05539132, -0.3799832 , -0.42401052,
         0.6005275 , -0.20597187, -0.09506731,  0.10353265,  0.44307077,
        -0.19796014,  0.31660974, -0.5254177 ,  0.96034443,  0.52602947,
         0.7290918 , -0.07997223, -0.3109342 , -0.58486176, -0.14334117,
         0.48111847, -0.7171643 ,  0.69819254,  0.00287792,  0.8008081 ,
         0.07767388,  0.36005935,  0.744768  ,  0.01837945, -0.01578468,
        -0.1760636 , -0.25540644, -0.14532836,  0.0

In [ ]:
review_scores = pd.DataFrame(d2v_model.dv.vocab.keys())
review_scores.columns = ['reviews']
# review_scores['vectors'] = review_scores.words.apply(lambda x: word_vectors.wv[f'{x}'])
# review_scroes['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
# review_scores.cluster = words.cluster.apply(lambda x: x[0])
# review_scores['cluster_value'] = [1 if i==0 else -1 for i in words.cluster]
# review_scores['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
# review_scores['sentiment_coeff'] = words.closeness_score * words.cluster_value
review_scores